<a href="https://colab.research.google.com/github/dipanjanS/nlp_workshop_odsc19/blob/master/Module03%20-%20Text%20Understanding/Project%20-%20Building%20NER%20Taggers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/dataset/dataset.csv')
df

,row_id,subject_id,hadm_id,icustay_id,startdate,enddate,drug_type,drug,drug_name_poe,drug_name_generic,formulary_drug_cd,gsn,ndc,prod_strength,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,route
0,32600,42458,159647,NaN,2146-07-21 00:00:00,2146-07-22 00:00:00,MAIN,Pneumococcal Vac Polyvalent,Pneumococcal Vac Polyvalent,PNEUMOcoccal Vac Polyvalent,PNEU25I,48548.0,6494300.0,25mcg/0.5mL Vial,0.5,mL,1,VIAL,IM
1,32601,42458,159647,NaN,2146-07-21 00:00:00,2146-07-22 00:00:00,MAIN,Bisacodyl,Bisacodyl,Bisacodyl,BISA5,2947.0,536338101.0,5 mg Tab,10,mg,2,TAB,PO
2,32602,42458,159647,NaN,2146-07-21 00:00:00,2146-07-22 00:00:00,MAIN,Bisacodyl,Bisacodyl,Bisacodyl (Rectal),BISA10R,2944.0,574705050.0,10mg Suppository,10,mg,1,SUPP,PR
3,32603,42458,159647,NaN,2146-07-21 00:00:00,2146-07-22 00:00:00,MAIN,Senna,Senna,Senna,SENN187,19964.0,904516561.0,1 Tablet,1,TAB,1,TAB,PO
4,32604,42458,159647,NaN,2146-07-21 00:00:00,2146-07-21 00:00:00,MAIN,Docusate Sodium (Liquid),Docusate Sodium (Liquid),Docusate Sodium (Liquid),DOCU100L,3017.0,121054410.0,100mg UD Cup,100,mg,1,UDCUP,PO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10393,3609913,42430,100969,210474.0,2142-11-29 00:00:00,2142-11-30 00:00:00,MAIN,Sodium Chloride 0.9% Flush,Sodium Chloride 0.9% Flush,Sodium Chloride 0.9% Flush,NACLFLUSH,NaN,0.0,Syringe,3,mL,0.6,SYR,IV
10394,3609914,42430,100969,210474.0,2142-11-30 00:00:00,2142-11-30 00:00:00,MAIN,Acetaminophen,Acetaminophen,Acetaminophen (Rectal),ACET650R,4478.0,713016550.0,650mg Supp,650,mg,1,SUPP,PR
10395,3609915,42430,100969,NaN,2142-11-26 00:00:00,2142-11-27 00:00:00,BASE,0.9% Sodium Chloride,NaN,NaN,NS1000,1210.0,338004904.0,1000mL Bag,1000,mL,1,BAG,IV
10396,3609916,42430,100969,NaN,2142-11-26 00:00:00,2142-11-27 00:00:00,BASE,D5W,NaN,NaN,HEPBASE,NaN,0.0,HEPARIN BASE,250,mL,250,mL,IV


In [ ]:
df.T

,0,1,2,3,4,5,6,7,8,9,...,10388,10389,10390,10391,10392,10393,10394,10395,10396,10397
row_id,32600,32601,32602,32603,32604,32605,32606,32607,32608,32609,...,3609908,3609909,3609910,3609911,3609912,3609913,3609914,3609915,3609916,3609917
subject_id,42458,42458,42458,42458,42458,42458,42458,42458,42458,42458,...,42430,42430,42430,42430,42430,42430,42430,42430,42430,42430
hadm_id,159647,159647,159647,159647,159647,159647,159647,159647,159647,159647,...,100969,100969,100969,100969,100969,100969,100969,100969,100969,100969
icustay_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,210474.0,210474.0,210474.0,210474.0,210474.0,210474.0,210474.0,NaN,NaN,NaN
startdate,2146-07-21 00:00:00,2146-07-21 00:00:00,2146-07-21 00:00:00,2146-07-21 00:00:00,2146-07-21 00:00:00,2146-07-21 00:00:00,2146-07-21 00:00:00,2146-07-21 00:00:00,2146-07-21 00:00:00,2146-07-21 00:00:00,...,2142-11-28 00:00:00,2142-11-28 00:00:00,2142-11-28 00:00:00,2142-11-28 00:00:00,2142-11-29 00:00:00,2142-11-29 00:00:00,2142-11-30 00:00:00,2142-11-26 00:00:00,2142-11-26 00:00:00,2142-11-26 00:00:00
enddate,2146-07-22 00:00:00,2146-07-22 00:00:00,2146-07-22 00:00:00,2146-07-22 00:00:00,2146-07-21 00:00:00,2146-07-22 00:00:00,2146-07-22 00:00:00,2146-07-22 00:00:00,2146-07-22 00:00:00,2146-07-22 00:00:00,...,2142-11-28 00:00:00,2142-11-28 00:00:00,2142-11-28 00:00:00,2142-11-30 00:00:00,2142-11-30 00:00:00,2142-11-30 00:00:00,2142-11-30 00:00:00,2142-11-27 00:00:00,2142-11-27 00:00:00,2142-11-28 00:00:00
drug_type,MAIN,MAIN,MAIN,MAIN,MAIN,MAIN,MAIN,MAIN,MAIN,MAIN,...,MAIN,MAIN,MAIN,MAIN,MAIN,MAIN,MAIN,BASE,BASE,BASE
drug,Pneumococcal Vac Polyvalent,Bisacodyl,Bisacodyl,Senna,Docusate Sodium (Liquid),Heparin,Simvastatin,Aspirin,Donepezil,Gabapentin,...,Ipratropium Bromide Neb,Midazolam,Insulin,Lorazepam,Pneumococcal Vac Polyvalent,Sodium Chloride 0.9% Flush,Acetaminophen,0.9% Sodium Chloride,D5W,0.83% Sodium Chloride
drug_name_poe,Pneumococcal Vac Polyvalent,Bisacodyl,Bisacodyl,Senna,Docusate Sodium (Liquid),Heparin,Simvastatin,Aspirin,Donepezil,Gabapentin,...,Ipratropium Bromide Neb,Midazolam,Insulin,Lorazepam,Pneumococcal Vac Polyvalent,Sodium Chloride 0.9% Flush,Acetaminophen,NaN,NaN,NaN
drug_name_generic,PNEUMOcoccal Vac Polyvalent,Bisacodyl,Bisacodyl (Rectal),Senna,Docusate Sodium (Liquid),Heparin Sodium,Simvastatin,Aspirin,Donepezil,Gabapentin,...,Ipratropium Bromide Neb,Midazolam,Insulin - Sliding Scale,Lorazepam,PNEUMOcoccal Vac Polyvalent,Sodium Chloride 0.9% Flush,Acetaminophen (Rectal),NaN,NaN,NaN


In [ ]:
df = df.fillna(method='ffill')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10398 entries, 0 to 10397
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   row_id             10398 non-null  int64  
 1   subject_id         10398 non-null  int64  
 2   hadm_id            10398 non-null  int64  
 3   icustay_id         10383 non-null  float64
 4   startdate          10398 non-null  object 
 5   enddate            10398 non-null  object 
 6   drug_type          10398 non-null  object 
 7   drug               10398 non-null  object 
 8   drug_name_poe      10398 non-null  object 
 9   drug_name_generic  10398 non-null  object 
 10  formulary_drug_cd  10398 non-null  object 
 11  gsn                10398 non-null  float64
 12  ndc                10398 non-null  float64
 13  prod_strength      10398 non-null  object 
 14  dose_val_rx        10398 non-null  object 
 15  dose_unit_rx       10398 non-null  object 
 16  form_val_disp      103

In [ ]:
df.T

,0,1,2,3,4,5,6,7,8,9,...,10388,10389,10390,10391,10392,10393,10394,10395,10396,10397
row_id,32600,32601,32602,32603,32604,32605,32606,32607,32608,32609,...,3609908,3609909,3609910,3609911,3609912,3609913,3609914,3609915,3609916,3609917
subject_id,42458,42458,42458,42458,42458,42458,42458,42458,42458,42458,...,42430,42430,42430,42430,42430,42430,42430,42430,42430,42430
hadm_id,159647,159647,159647,159647,159647,159647,159647,159647,159647,159647,...,100969,100969,100969,100969,100969,100969,100969,100969,100969,100969
icustay_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,210474.0,210474.0,210474.0,210474.0,210474.0,210474.0,210474.0,210474.0,210474.0,210474.0
startdate,2146-07-21 00:00:00,2146-07-21 00:00:00,2146-07-21 00:00:00,2146-07-21 00:00:00,2146-07-21 00:00:00,2146-07-21 00:00:00,2146-07-21 00:00:00,2146-07-21 00:00:00,2146-07-21 00:00:00,2146-07-21 00:00:00,...,2142-11-28 00:00:00,2142-11-28 00:00:00,2142-11-28 00:00:00,2142-11-28 00:00:00,2142-11-29 00:00:00,2142-11-29 00:00:00,2142-11-30 00:00:00,2142-11-26 00:00:00,2142-11-26 00:00:00,2142-11-26 00:00:00
enddate,2146-07-22 00:00:00,2146-07-22 00:00:00,2146-07-22 00:00:00,2146-07-22 00:00:00,2146-07-21 00:00:00,2146-07-22 00:00:00,2146-07-22 00:00:00,2146-07-22 00:00:00,2146-07-22 00:00:00,2146-07-22 00:00:00,...,2142-11-28 00:00:00,2142-11-28 00:00:00,2142-11-28 00:00:00,2142-11-30 00:00:00,2142-11-30 00:00:00,2142-11-30 00:00:00,2142-11-30 00:00:00,2142-11-27 00:00:00,2142-11-27 00:00:00,2142-11-28 00:00:00
drug_type,MAIN,MAIN,MAIN,MAIN,MAIN,MAIN,MAIN,MAIN,MAIN,MAIN,...,MAIN,MAIN,MAIN,MAIN,MAIN,MAIN,MAIN,BASE,BASE,BASE
drug,Pneumococcal Vac Polyvalent,Bisacodyl,Bisacodyl,Senna,Docusate Sodium (Liquid),Heparin,Simvastatin,Aspirin,Donepezil,Gabapentin,...,Ipratropium Bromide Neb,Midazolam,Insulin,Lorazepam,Pneumococcal Vac Polyvalent,Sodium Chloride 0.9% Flush,Acetaminophen,0.9% Sodium Chloride,D5W,0.83% Sodium Chloride
drug_name_poe,Pneumococcal Vac Polyvalent,Bisacodyl,Bisacodyl,Senna,Docusate Sodium (Liquid),Heparin,Simvastatin,Aspirin,Donepezil,Gabapentin,...,Ipratropium Bromide Neb,Midazolam,Insulin,Lorazepam,Pneumococcal Vac Polyvalent,Sodium Chloride 0.9% Flush,Acetaminophen,Acetaminophen,Acetaminophen,Acetaminophen
drug_name_generic,PNEUMOcoccal Vac Polyvalent,Bisacodyl,Bisacodyl (Rectal),Senna,Docusate Sodium (Liquid),Heparin Sodium,Simvastatin,Aspirin,Donepezil,Gabapentin,...,Ipratropium Bromide Neb,Midazolam,Insulin - Sliding Scale,Lorazepam,PNEUMOcoccal Vac Polyvalent,Sodium Chloride 0.9% Flush,Acetaminophen (Rectal),Acetaminophen (Rectal),Acetaminophen (Rectal),Acetaminophen (Rectal)


In [ ]:
df['row_id'].nunique(), df['drug'].nunique(), df['drug_type'].nunique(), df['drug_name_generic'].nunique()

(10398, 592, 3, 473)

In [ ]:
import re
import string

def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree

def tokenization(text):
    tokens = re.split('W+',text)
    return tokens

def remove_stopwords(text):
    stopwords = nltk.corpus.stopwords.words('english')
    stopwords[0:10]
    output= [i for i in text if i not in stopwords]
    return output

def lemmatizer(text):
    from nltk.stem import WordNetLemmatizer

    wordnet_lemmatizer = WordNetLemmatizer()
    lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return lemm_text
    
def preprocess(text):
    text = remove_punctuation(text)
    text = tokenization(text)
    text = remove_stopwords(text)
    text = lemmatizer(text)
    return text


In [ ]:
agg_func = lambda s: [(w, p, t) for w, p, t in zip(s['drug_name_generic'].values.tolist(), 
                                                   s['drug'].values.tolist(), 
                                                   s['drug_type'].values.tolist())]

In [ ]:
grouped_df = df.groupby('drug').apply(agg_func)
grouped_df

drug
*NF* Ertapenem Sodium    [(OxycoDONE (Immediate Release) , *NF* Ertapen...
*NF* Rasburicase         [(Methadone, *NF* Rasburicase, MAIN), (Methado...
0.45% Sodium Chloride    [(Baclofen, 0.45% Sodium Chloride, BASE), (Neu...
0.83% Sodium Chloride    [(Acetaminophen (Rectal), 0.83% Sodium Chlorid...
0.9% Sodium Chloride     [(Morphine Sulfate (Syringe), 0.9% Sodium Chlo...
                                               ...                        
sodium bicarb                  [(Sodium Bicarbonate, sodium bicarb, MAIN)]
traMADOL                 [(traMADOL, traMADOL, MAIN), (traMADOL, traMAD...
traZODONE                [(traZODONE, traZODONE, MAIN), (traZODONE, tra...
traZODONE HCl            [(traZODONE HCl, traZODONE HCl, MAIN), (traZOD...
tucks                          [(Tucks Hemorrhoidal Oint 1%, tucks, MAIN)]
Length: 592, dtype: object

In [ ]:
grouped_df.shape

(592,)

In [ ]:
[(preprocess(i[0][0])) for i in grouped_df]

[['OxycoDONE Immediate Release '],
 ['Methadone'],
 ['Baclofen'],
 ['Acetaminophen Rectal'],
 ['Morphine Sulfate Syringe'],
 ['Morphine Sulfate Syringe'],
 ['Morphine Sulfate Syringe'],
 ['Morphine Sulfate Syringe'],
 ['Scopolamine Patch'],
 ['Acebutolol HCl'],
 ['AcetaZOLamide'],
 ['Lorazepam'],
 ['Acetaminophen'],
 ['Acetaminophen Liquid'],
 ['Acetaminophen IV'],
 ['Acetaminophen wCodeine'],
 ['Acetylcysteine 20'],
 ['NF Ketorolac 05 Ophth Soln'],
 ['Metformin'],
 ['Adenosine'],
 ['Albumin 5 125 g'],
 ['Albumin 25 125 g'],
 ['Albumin 25 125g  50mL'],
 ['Albumin 25 125gm'],
 ['Albumin 5 125 g'],
 ['Albumin 5 125 g'],
 ['Albumin 5 125g  250mL'],
 ['Albumin 5 25 g'],
 ['Albumin 5 25g  500mL'],
 ['Metronidazole'],
 ['Albuterol Inhaler'],
 ['Albuterol 0083 Neb Soln'],
 ['Albuterol Inhaler'],
 ['Albuterol Inhaler'],
 ['Albuterol Neb Soln'],
 ['Albuterol SulfateIpratropium'],
 ['Alendronate Sodium'],
 ['Allopurinol'],
 ['Alprazolam'],
 ['Alteplase Catheter Clearance'],
 ['Aluminum Hydroxide

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

text = [(preprocess(i[0][0])) for i in grouped_df]

X_train, X_test, y_train, y_test = train_test_split(text[:(len(text) // 2)], text[(len(text) // 2): ], test_size=0.25, random_state=42)
X_train, X_test

([['Calcium Acetate'],
  ['Morphine Sulfate 26mg'],
  ['Bismuth Subsalicylate'],
  ['Acetaminophen wCodeine'],
  ['AcetaZOLamide'],
  ['Lorazepam'],
  ['Metoprolol'],
  ['Clonazepam'],
  ['Allopurinol'],
  ['Acetylcysteine 20'],
  ['Metronidazole'],
  ['NF Cromolyn Sodium 4 Opth'],
  ['Hepatitis B Immune Globulin NabiHB'],
  ['Metronidazole'],
  ['CarbidopaLevodopa 25250'],
  ['Benzocaine 20 Spray'],
  ['Morphine Sulfate Syringe'],
  ['Digoxin'],
  ['CloniDINE'],
  ['Fluocinonide 005 Ointment'],
  ['Ipratropium Bromide Neb'],
  ['Metformin'],
  ['Divalproex Sod Sprinkles'],
  ['Metoprolol'],
  ['Albumin 5 125g  250mL'],
  ['Capsaicin 0025'],
  ['Gelclair'],
  ['Creon 10'],
  ['Docusate Sodium Liquid'],
  ['Chlorhexidine Gluconate'],
  ['Golytely'],
  ['Erythromycin 05 Ophth Oint'],
  ['Ascorbic Acid Liquid'],
  ['HYDROmorphone Dilaudid'],
  ['Dexamethasone Sod Phosphate'],
  ['Glipizide'],
  ['Alprazolam'],
  ['Dronabinol'],
  ['Ibuprofen Suspension'],
  ['Aspirin EC'],
  ['Ciprofloxac

In [ ]:
!pip install sklearn-crfsuite

     |████████████████████████████████| 965 kB 5.2 MB/s 


In [ ]:
import sklearn_crfsuite

crf = sklearn_crfsuite.CRF(algorithm='lbfgs',
                           c1=0.1,
                           c2=0.1,
                           max_iterations=100,
                           all_possible_transitions=True,
                           verbose=True)

In [ ]:
#@title
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

import pandas as pd

df = pd.read_csv('https://github.com/dipanjanS/nlp_workshop_dhs18/raw/master/Unit%2008%20-%20Project%206%20-%20Build%20your%20NER%20Tagger/ner_dataset.csv.gz', compression='gzip', encoding='ISO-8859-1')
df.info()

df.T

df = df.fillna(method='ffill')
df.info()

df.T

df['Sentence #'].nunique(), df.Word.nunique(), df.POS.nunique(), df.Tag.nunique()

df.Tag.value_counts()

def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

agg_func = lambda s: [(w, p, t) for w, p, t in zip(s['Word'].values.tolist(), 
                                                   s['POS'].values.tolist(), 
                                                   s['Tag'].values.tolist())]

grouped_df = df.groupby('Sentence #').apply(agg_func)

print(grouped_df[grouped_df.index == 'Sentence: 1'].values)

grouped_df.shape

sentences = [s for s in grouped_df]
sentences[0]

sent2features(sentences[0][5:7])

sent2labels(sentences[0][5:7])

from sklearn.model_selection import train_test_split
import numpy as np

X = np.array([sent2features(s) for s in sentences])
y = np.array([sent2labels(s) for s in sentences])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
X_train.shape, X_test.shape

!pip install sklearn-crfsuite

import sklearn_crfsuite

crf = sklearn_crfsuite.CRF(algorithm='lbfgs',
                           c1=0.1,
                           c2=0.1,
                           max_iterations=100,
                           all_possible_transitions=True,
                           verbose=True)



In [ ]:
crf.fit(X_train, y_train)

loading training data to CRFsuite: 100%|██████████| 35969/35969 [00:21<00:00, 1688.93it/s]



Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 133629
Seconds required: 3.083

L-BFGS optimization
c1: 0.100000
c2: 0.100000
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=3.54  loss=1264028.26 active=132637 feature_norm=1.00
Iter 2   time=3.77  loss=994059.01 active=131294 feature_norm=4.42
Iter 3   time=1.86  loss=776413.87 active=125970 feature_norm=3.87
Iter 4   time=9.31  loss=422143.40 active=127018 feature_norm=3.24
Iter 5   time=2.22  loss=355775.44 active=129029 feature_norm=4.04
Iter 6   time=1.87  loss=264125.22 active=124046 feature_norm=6.10
Iter 7   time=1.93  loss=222304.71 active=117183 feature_norm=7.69
Iter 8   time=1.86  loss=197827.17 active=110838 feature_norm=8.75
Iter 9   time=1.88  loss=176877.92 active=105650 feature_norm

AttributeError: ignored

TypeError: ignored

In [ ]:
y_pred = crf.predict(X_test)
print(y_pred[0])

['O', 'O', 'O', 'O', 'B-per', 'I-per', 'O', 'B-org', 'O', 'O', 'B-gpe', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [ ]:
print(y_test[0])

['O', 'O', 'O', 'O', 'B-per', 'I-per', 'O', 'B-org', 'O', 'O', 'B-gpe', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [ ]:
from sklearn_crfsuite import metrics as crf_metrics

labels = list(crf.classes_)
labels.remove('O')

In [ ]:
print(crf_metrics.flat_classification_report(y_test, y_pred, labels=labels))

              precision    recall  f1-score   support

       B-org       0.81      0.73      0.77      5116
       B-per       0.85      0.84      0.84      4239
       I-per       0.85      0.90      0.88      4273
       B-geo       0.86      0.91      0.89      9403
       I-geo       0.81      0.80      0.81      1826
       B-tim       0.93      0.89      0.91      5095
       I-org       0.82      0.79      0.80      4195
       B-gpe       0.97      0.94      0.96      3961
       I-tim       0.84      0.81      0.82      1604
       B-nat       0.50      0.24      0.32        55
       B-eve       0.51      0.33      0.40        80
       B-art       0.36      0.14      0.20       102
       I-art       0.24      0.07      0.10        90
       I-eve       0.45      0.19      0.27        74
       I-gpe       0.86      0.53      0.66        36
       I-nat       0.57      0.22      0.32        18

   micro avg       0.86      0.85      0.86     40167
   macro avg       0.70   